In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from plot_utils import movingaverage
from main_FT_input_num import load_data_and_embeddings
import torch
from utils import accuracy
from tqdm.notebook import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_name = 'cifar10'
embed = 'OpenAI'
plot_linear = True
cvxpy = False
if data_name == 'ECG-signal':
    cvxpy = True
sdim = 100
train_num_str = '_TN_f1'
plot_full = False
eps = True
Hidden = 50
shuffle = True
skip = False
Hidden_str = '_Hidden{}'.format(Hidden)
Hidden_str_lasso = '_Hidden500'
if data_name=='cola' and Hidden == 10:
    Hidden_str = ''
shuffle_str = ''
if shuffle == True:
    shuffle_str = '_shuffle'
skip_str = ''
if skip == True:
    skip_str = '_skip'
solver_str = ''
if cvxpy == True:
    solver_str = '_cvxpy'
eps_str = ''
if eps == True:
    eps_str = '_eps1e-08'
sdim_str = ''
if sdim!=100:
    sdim_str = '_sdim{}'.format(sdim)
seed = 1
aug_str = '_aug'
str_bundle = sdim_str+shuffle_str+skip_str+solver_str+eps_str
container = np.load('results/FT_{}_T_f1{}_{}_cvx{}{}_NT5_seed{}.npz'.format(data_name,train_num_str,embed,Hidden_str,str_bundle,seed),allow_pickle=True)
result_dict_cvx = container['result_dict'].item()
sdim_str_alt = ''#'_sdim-1'
str_bundle = sdim_str_alt+shuffle_str+skip_str
container = np.load('results/FT_{}_T_f1{}_{}_noncvx{}{}_NT5_seed{}.npz'.format(data_name,train_num_str,embed,Hidden_str,str_bundle,seed),allow_pickle=True)
result_dict_noncvx = container['result_dict'].item()

In [ ]:
data_path = './data/'
if 'ECG' in data_name:
    data_path = './data/ECG/'

if data_name == 'IMDB':
    Bert_embeddings, Bert_labels = load_data_and_embeddings("IMDB-Bert-full", data_path=data_path)
    OpenAI_embeddings, OpenAI_labels = load_data_and_embeddings("IMDB-OpenAI-full", data_path=data_path)
elif data_name == 'Amazon':
    Bert_embeddings, Bert_labels = load_data_and_embeddings("Amazon-Bert-30K", data_path=data_path)
    OpenAI_embeddings, OpenAI_labels = load_data_and_embeddings("Amazon-OpenAI-30K", data_path=data_path)
elif data_name == 'cvx-forum':
    Bert_embeddings, Bert_labels = load_data_and_embeddings("cvx-forum-Bert-full", data_path=data_path)
    OpenAI_embeddings, OpenAI_labels = load_data_and_embeddings("cvx-forum-OpenAI-full", data_path=data_path)
elif data_name == 'cola':
    Bert_embeddings, Bert_labels = load_data_and_embeddings("glue-cola-Bert-full", data_path=data_path)
    OpenAI_embeddings, OpenAI_labels = load_data_and_embeddings("glue-cola-OpenAI-full", data_path=data_path)
elif data_name == 'qqp':
    Bert_embeddings, Bert_labels = load_data_and_embeddings("glue-qqp-Bert-50K", data_path=data_path)
    OpenAI_embeddings, OpenAI_labels = load_data_and_embeddings("glue-qqp-OpenAI-50K", data_path=data_path)
elif data_name == 'ECG-signal':
    _, labels = load_data_and_embeddings("ECG-report", data_path=data_path)
    embeddings, _ = load_data_and_embeddings("ECG-signal", data_path=data_path)
    embeddings = torch.clamp(embeddings,max=1)*0.15
    Bert_embeddings, Bert_labels = embeddings.clone(), labels.copy()
    OpenAI_embeddings, OpenAI_labels = embeddings.clone(), labels.copy()
elif data_name == 'ECG-report':
    embeddings, labels = load_data_and_embeddings("ECG-report", data_path=data_path)
    embeddings = torch.nan_to_num(embeddings)
    Bert_embeddings, Bert_labels = embeddings.clone(), labels.copy()
    OpenAI_embeddings, OpenAI_labels = embeddings.clone(), labels.copy()
elif data_name == 'ECG-sr':
    embeddings_signal, _ = load_data_and_embeddings("ECG-signal", data_path=data_path)
    embeddings_report, labels = load_data_and_embeddings("ECG-report", data_path=data_path)
    embeddings_signal = torch.clamp(embeddings_signal,max=1)*0.15
    embeddings_report = torch.nan_to_num(embeddings_report)
    embeddings = torch.cat([embeddings_signal,embeddings_report],dim=1)
    Bert_embeddings, Bert_labels = embeddings.clone(), labels.copy()
    OpenAI_embeddings, OpenAI_labels = embeddings.clone(), labels.copy()
elif data_name == 'mnist':
    container = np.load('{}mnist_transformed.npz'.format(data_path))
    training_data_np = container['train_data'].reshape([60000,-1])
    training_labels_np = container['train_label']
    test_data_np = container['test_data'].reshape([10000,-1])
    test_labels_np = container['test_label']
    embeddings = np.concatenate([training_data_np, test_data_np],axis=0)
    labels = np.concatenate([training_labels_np, test_labels_np])
    index1 = np.where(labels==0)
    index2 = np.where(labels==1)
    index = np.concatenate([index1[0],index2[0]])
    embeddings = embeddings[index,:]
    labels = labels[index]
    embeddings = torch.tensor(embeddings).float()
    Bert_embeddings, Bert_labels = embeddings.clone(), labels.copy()
    OpenAI_embeddings, OpenAI_labels = embeddings.clone(), labels.copy()
elif data_name == 'cifar10':
    container = np.load('{}cifar10_transformed.npz'.format(data_path))
    training_data_np = container['train_data'].reshape([50000,-1])
    training_labels_np = container['train_label']
    test_data_np = container['test_data'].reshape([10000,-1])
    test_labels_np = container['test_label']
    embeddings = np.concatenate([training_data_np, test_data_np],axis=0)
    labels = np.concatenate([training_labels_np, test_labels_np])
    index1 = np.where(labels==0)
    index2 = np.where(labels==1)
    index = np.concatenate([index1[0],index2[0]])
    embeddings = embeddings[index,:]
    labels = labels[index]
    embeddings = torch.tensor(embeddings).float()
    Bert_embeddings, Bert_labels = embeddings.clone(), labels.copy()
    OpenAI_embeddings, OpenAI_labels = embeddings.clone(), labels.copy()
elif data_name == 'ECG-signal-mfcc':
    _, labels = load_data_and_embeddings("ECG-report", data_path=data_path)
    embeddings = np.load('{}signal_mfcc.npy'.format(data_path))
    embeddings = torch.tensor(embeddings).float()
    Bert_embeddings, Bert_labels = embeddings.clone(), labels.copy()
    OpenAI_embeddings, OpenAI_labels = embeddings.clone(), labels.copy()
elif data_name == 'ECG-sr-mfcc':
    embeddings = np.load('{}signal_mfcc.npy'.format(data_path))
    embeddings_signal = torch.tensor(embeddings).float()
    embeddings_report, labels = load_data_and_embeddings("ECG-report", data_path=data_path)
    embeddings_report = torch.nan_to_num(embeddings_report)
    embeddings = torch.cat([embeddings_signal,embeddings_report],dim=1)
    Bert_embeddings, Bert_labels = embeddings.clone(), labels.copy()
    OpenAI_embeddings, OpenAI_labels = embeddings.clone(), labels.copy()

if plot_linear:
    n = Bert_embeddings.shape[0]
    num_train = n//2

    index = np.arange(n)
    np.random.seed(2)
    np.random.shuffle(index)
    Bert_train = Bert_embeddings[index[:num_train]].detach().numpy()
    label_train = Bert_labels[index[:num_train]]
    OpenAI_train = OpenAI_embeddings[index[:num_train]].detach().numpy()

    Bert_test = Bert_embeddings[index[num_train:n]].detach().numpy()
    label_test = Bert_labels[index[num_train:n]]
    OpenAI_test = OpenAI_embeddings[index[num_train:n]].detach().numpy()

    label_train = np.array(label_train)*2-1
    label_test = np.array(label_test)*2-1

    training_data_np = Bert_train.copy()
    training_labels_np = label_train.copy()
    test_data_np = Bert_test.copy()
    test_labels_np = label_test.copy()

In [ ]:
methods = ['Gaussian', 'Geometric_Algebra']
num_trial = 5
beta_list = [1e-3,1e-4,1e-5,1e-6]
lr_list = [1e-2,1e-3,1e-4]

def compute_test_acc_cvx(result_dict_cvx,input_num_list,mode='early_stop', patience=5, threshold=1e-2,coef=1):
    time_array = np.zeros([num_trial, 2,len(beta_list)])
    test_acc_array = np.zeros([len(input_num_list),2,len(beta_list),num_trial])
    t_array = np.zeros([len(input_num_list)])
    for i0, input_num in enumerate(input_num_list):
        for i in range(num_trial):
            for j1, method in enumerate(methods):
                for j2, beta in enumerate(beta_list):
                    key = '{}_{}_{}_{}'.format(input_num,method,beta,i)
                    metrics = result_dict_cvx[key]
                    time_array[i,j1,j2] =  metrics.time[-1]
                    acc_array = np.array(metrics.test_accuracy)
                    if mode=='early_stop':
                        idx = early_stop(acc_array,patience=patience, threshold=threshold)
                    elif mode=='best':
                        idx = np.argmax(acc_array)
                    time_array[i,j1,j2] = metrics.time[idx]
                    
        t0 = coef*np.min(time_array[:,:,:])

        t_array[i0] = t0
        print(t0)
        for i in range(num_trial):
            for j1, method in enumerate(methods):
                for j2, beta in enumerate(beta_list):
                    key = '{}_{}_{}_{}'.format(input_num,method,beta,i)
                    metrics = result_dict_cvx[key]
                    try:
                        idx = np.where(metrics.time>=t0)[0][0]
                    except:
                        idx = -1
                    test_acc_array[i0,j1,j2,i] = metrics.test_accuracy[idx]
        
    return test_acc_array, t_array

def compute_test_acc_noncvx(result_dict_noncvx,t_array,input_num_list):
    time_array = np.zeros([num_trial, 2,len(beta_list)])
    test_acc_array_noncvx = np.zeros([len(input_num_list),len(lr_list),num_trial])
    for i0, input_num in enumerate(input_num_list):
        t0 = t_array[i0]
        for i in range(num_trial):
            for j1, lr in enumerate(lr_list):
                key = '{}_{}_{}'.format(input_num,lr,i)
                metrics = result_dict_noncvx[key]
                try:
                    idx = np.where(metrics['cum_time']>=t0)[0][0]+1
                except:
                    idx = -1
#                 idx = -1
                test_acc_array_noncvx[i0,j1,i] = metrics['test_acc'][idx]
    return test_acc_array_noncvx

def compute_test_acc_lasso(result_dict_lasso,input_num_list,beta_list):
    test_acc_array_lasso = np.zeros([len(input_num_list),len(beta_list),num_trial])
    for i0, input_num in enumerate(input_num_list):
        for i in range(num_trial):
            for j, beta in enumerate(beta_list):
                key = '{}_{}_{}'.format(input_num,beta,i)
                metrics = result_dict_lasso[key]
                test_acc_array_lasso[i0,j,i] =  metrics['test_acc']

    return test_acc_array_lasso

def early_stop(test_acc, patience=5, threshold=1e-2):
    n = len(test_acc)
    if n<patience:
        return n-1
    i = patience
    loss = 1-test_acc
    min_loss = loss[0]
    while i<n:
        if loss[i]>min_loss*(1+threshold) and np.max(loss[i-patience:i])<=min_loss*(1+threshold):
            return i
        i+=1
        min_loss = min(min_loss,loss[i-patience])
    return n-1

In [ ]:
# linear model
if plot_linear:
    from sklearn import linear_model
    num_trial = 5
    test_acc_array_linear = np.zeros([len(input_num_list),num_trial])
    beta = 1e-3
    for j in tqdm(range(num_trial)):
        for i, input_num in enumerate(input_num_list):
            index = np.arange(num_train)
            training_data_np_sub = training_data_np[index[:input_num]]
            training_labels_np_sub = training_labels_np[index[:input_num]]
            clf = linear_model.Lasso(alpha=beta)
            clf.fit(training_data_np_sub,training_labels_np_sub)
            logits = clf.predict(test_data_np)
            test_acc = accuracy(logits,test_labels_np)
            test_acc_array_linear[i,j] = test_acc



In [ ]:
input_num_list = [100,200,500,1000,2000,5000,10000,20000]
if data_name == 'Amazon':
    input_num_list = [100,200,500,1000,2000,5000,10000]
elif data_name == 'cola':
    input_num_list = [100,200,500,1000,2000]
if train_num_str == '_TN_f1':
    input_num_list = 200*np.arange(1,11)
elif train_num_str == '_TN_f2':
    input_num_list = 50*np.arange(4,11)
elif train_num_str == '_TN_f3':
    input_num_list = [5000,10000]
time_str = ''
mode='early_stop'
# mode='best'
time_str = '_early_stop'
patience = 3
threshold = 1e-2
if data_name == 'Amazon':
    threshold = 2e-2
elif data_name == 'qqp':
    threshold = 1e-3
elif data_name == 'mnist':
    threshold = 2e-1
beta_list_lasso = [1e-1,1e-2,1e-3,1e-4]
test_acc_array, t_array = compute_test_acc_cvx(result_dict_cvx,input_num_list,mode=mode,
                                                patience=patience,threshold=threshold)
test_acc_array_noncvx = compute_test_acc_noncvx(result_dict_noncvx,t_array,input_num_list)
if plot_lasso:
    test_acc_array_lasso = compute_test_acc_lasso(result_dict_lasso,input_num_list,beta_list_lasso)
if plot_polish:
    test_acc_array_polish = compute_test_acc_noncvx(result_dict_polish,result_dict_cvx,input_num_list,coef)
# if plot_full:
   

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ls = 'dotted'
scale = 1
for j1, method in enumerate(methods):
    target = np.max(test_acc_array[:,j1,:,:],axis=1)*100
    t_mean = np.mean(target,axis=1)
    t_std = np.sqrt(np.var(target,axis=1))
    ax.errorbar(input_num_list, t_mean, yerr=scale*t_std, label='CVX {}'.format(method),linestyle=ls,marker='o',markersize=4,capsize=4)

target = np.max(test_acc_array_noncvx,axis=1)
t_mean = np.mean(target,axis=1)
t_std = np.sqrt(np.var(target,axis=1))
ax.errorbar(input_num_list, t_mean, yerr=scale*t_std, label='nonCVX AdamW',linestyle=ls,marker='o',markersize=4,capsize=4)

if plot_linear:
    target = test_acc_array_linear*100
    t_mean = np.mean(target,axis=1)
    t_std = np.sqrt(np.var(target,axis=1))
    ax.errorbar(input_num_list, t_mean, yerr=scale*t_std, label='linear',linestyle=ls,marker='o',markersize=4,capsize=4)

ax.legend()
log=False
log_str = ''
if log:
    ax.set_xscale('log')
    log_str = '-log'
ax.set_xlabel('Training data amount')
ax.set_ylabel('Test accuracy')
str_bundle = time_str+train_num_str+shuffle_str+skip_str+solver_str+eps_str+aug_str
fig.savefig('figures/FT-input-num-{}-{}-Hidden{}{}.pdf'.format(data_name,embed,Hidden,str_bundle),bbox_inches='tight')

In [ ]:
input_num = 2000

def append_zero(np_array):
    return np.concatenate([[0],np_array])

number = 10
cmap = plt.get_cmap('jet')
    # colors = [cmap(i) for i in np.linspace(0, 1, number)]

fig, axes = plt.subplots(1,2,figsize=(10, 5))
for i0 in range(2):
    ax = axes[i0]
    index = 0

    for e, method in enumerate(methods):
        best_beta_idx = np.argmax(test_acc_array[-1,e,:,0])
        beta = beta_list[best_beta_idx]
        train_acc_list = []
        for i in range(num_trial):
            key = '{}_{}_{}_{}'.format(input_num,method,beta,i)
            metrics = result_dict_cvx[key]
            if i0==0:
                train_acc = metrics.train_accuracy
            else:
                train_acc = metrics.test_accuracy
            train_acc_list.append(train_acc)
        min_train_acc_len = np.min([len(x) for x in train_acc_list])
        train_acc_array = np.zeros([num_trial, min_train_acc_len])
        for i in range(num_trial):
            train_acc_array[i,:] = train_acc_list[i][:min_train_acc_len]
        target = train_acc_array*100
        t_mean = np.mean(target,axis=0)
        t_std = np.sqrt(np.var(target,axis=0))
        cum_time = metrics.time[:min_train_acc_len]
        ax.plot(cum_time, t_mean, label='CVX {}'.format(method))
        ax.fill_between(cum_time, t_mean - scale*t_std, t_mean + scale*t_std, alpha=0.2)
        index+=1

    x_lim = cum_time[-1]

    best_lr_idx = np.argmax(test_acc_array_noncvx[-1,:,0])
    lr = lr_list[best_lr_idx]
    train_acc_list = []
    for i in range(num_trial):
        key = '{}_{}_{}'.format(input_num,lr,i)
        if i0==0:
            train_acc = result_dict_noncvx[key]['train_acc']
        else:
            train_acc = result_dict_noncvx[key]['test_acc']
        train_acc_list.append(train_acc)
    min_train_acc_len = np.min([len(x) for x in train_acc_list])
    train_acc_array = np.zeros([num_trial, min_train_acc_len])
    for i in range(num_trial):
        train_acc_array[i,:] = train_acc_list[i][:min_train_acc_len]
    target = train_acc_array
    t_mean = append_zero(np.mean(target,axis=0))
    t_std = append_zero(np.sqrt(np.var(target,axis=0)))
    cum_time = append_zero(result_dict_noncvx[key]['cum_time'][:min_train_acc_len])
    ax.plot(cum_time, t_mean, label='nonCVX AdamW')
    ax.fill_between(cum_time, t_mean - scale*t_std, t_mean + scale*t_std, alpha=0.2)
    index+=1


    x_lim = min(x_lim, cum_time[-1])
    if i0==0:
        title_str = 'Training'
    else:
        title_str = 'Test'
    ax.set_title('{} Accuracy'.format(title_str))
    ax.set_xlabel('Cumulative Time (s)')
    ax.set_ylabel('Accuracy (%)')
    theta=1.2
    x_lim = max(t_array[-1]*theta,0.5)
    ax.set_xlim([0,x_lim])
    if i0==1:
        if data_name == 'ECG-signal' or data_name == 'qqp':
            y_lims = [60,80]
        elif data_name == 'cola':
            y_lims = [60,80]
        elif data_name == 'cifar10':
            y_lims = [70,90]
        else:
            y_lims = [80,100]
    else:
        if data_name == 'ECG-signal' or data_name == 'qqp':
            y_lims = [60,100]
        elif data_name == 'IMDB':
            y_lims = [70,100]
        else:
            y_lims = [80,100]
    ax.set_ylim(y_lims)
    ax.legend()
str_bundle = time_str+train_num_str+shuffle_str+skip_str+solver_str+eps_str+aug_str+'_IN{}'.format(input_num)
fig.savefig('figures/TR-input-num-{}-{}-Hidden{}{}.pdf'.format(data_name,embed,Hidden,str_bundle),bbox_inches='tight')